In [ ]:
!pip install python-telegram-bot==13.15

In [ ]:
import os
import torch
import spacy
import nltk
from transformers import BertTokenizer, BertForSequenceClassification
from telegram.ext import Updater, MessageHandler, Filters, CommandHandler
from telegram import ParseMode
import threading

nltk.download("stopwords")
nlp = spacy.load("en_core_web_sm")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
def classify_text_with_bert(text):
    from nltk.corpus import stopwords
    stop_words = set(stopwords.words("english"))

    article_clean = " ".join([
        token.lemma_ for token in nlp(text.lower())
        if token.is_alpha and token.text not in stop_words
    ])

    tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
    model = BertForSequenceClassification.from_pretrained("./bert_model")
    model.eval()

    inputs = tokenizer(article_clean, truncation=True, padding=True, return_tensors="pt", max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
        prediction = torch.argmax(outputs.logits, dim=1).item()

    return "REAL" if prediction == 1 else "FAKE"


In [ ]:
TELEGRAM_TOKEN = "7620382002:AAF5wV5PzPxnKFlm6CDkU7epnt_qI__qwu0"

def start(update, context):
    update.message.reply_text(
        "Привет! Пришли .txt файл с текстом статьи на английском, и я определю фейковая она или нет.\n"
        "Файл должен быть в формате .txt и содержать только текст статьи."
    )

def handle_plain_text(update, context):
    update.message.reply_text("Пожалуйста, отправь файл, я пока не умею анализировать сообщения(")

def handle_document(update, context):
    file = update.message.document

    if file.mime_type != 'text/plain':
        update.message.reply_text("Пожалуйста, отправь .txt файл.")
        return

    update.message.reply_text("Файл получен. Анализирую...")

    file_path = f"{file.file_id}.txt"
    file.get_file().download(file_path)

    try:
        with open(file_path, "r", encoding="utf-8") as f:
            text = f.read()

        result = classify_text_with_bert(text)
        update.message.reply_text(f"Результат анализа: *{result}*", parse_mode=ParseMode.MARKDOWN)

    except Exception as e:
        update.message.reply_text(f"Ошибка при анализе файла: {str(e)}")

    finally:
        if os.path.exists(file_path):
            os.remove(file_path)

def run_bot():
    updater = Updater(TELEGRAM_TOKEN, use_context=True)
    dp = updater.dispatcher

    dp.add_handler(CommandHandler("start", start))
    dp.add_handler(MessageHandler(Filters.document.mime_type("text/plain"), handle_document))
    dp.add_handler(MessageHandler(Filters.text & ~Filters.command, handle_plain_text))

    updater.start_polling()
    updater.idle()

threading.Thread(target=run_bot).start()